In [24]:
%pip install qdrant-client==1.7.0 
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
import random
import time
import os
import shutil

In [25]:
n_samples = 100_000
n_features = 2_000

data = np.random.randn(n_samples, n_features).astype(np.float64)
data[0][0], type(data[0][0])

(1.2321054320970832, numpy.float64)

In [27]:
def calculate_recall(true_ids, predicted_ids):
    true_positive = len(set(true_ids).intersection(predicted_ids))
    total_relevant = len(true_ids)
    recall = true_positive / total_relevant
    return recall

In [28]:
data_to_insert = []
for i, d in enumerate(data):
    point = models.PointStruct(
        id=i, vector=d, payload={"city": random.choice(["New York", "Berlin"])}
    )
    data_to_insert.append(point)
data_to_insert[0]

PointStruct(id=0, vector=[1.2321054320970832, -0.6849135173124598, -0.074726719699931, -1.0830340182103582, -0.9857315588705456, -0.44152642192309854, 1.039836878277819, -0.762135692676492, 1.3920494760281443, -0.36907373398165183, 1.5183329028131316, -1.7075136654000742, -0.2669228384523699, 0.25024747348417603, 0.012428448305860835, 0.16004265842965565, 1.8441082085901885, -0.10762225984762991, 1.2851305467160938, -0.9494611805885296, -0.10825563821279696, 1.5207013583652402, -0.23838316552043157, 0.7935620367489004, 0.15975376755244552, -0.2567062156415976, -0.7950495403065091, 0.04500671302915048, 0.9744506489494862, -0.6251148788678225, -0.3533746160104681, 0.8577475406915583, -0.3634359998488009, -0.49531700128735684, -0.09003135699345391, 0.15857653552045903, 0.3728226593221158, -0.8202052916728672, -1.7305039627274106, -0.858187532390304, 0.535232346402151, 1.4328256137636932, 0.7898075453223514, 0.012881155435706419, 0.7073210341846855, 0.9702494335117134, -0.2812319788994164,

In [29]:
if os.path.exists("data/qdrant_db"):
    shutil.rmtree("data/qdrant_db")

client = QdrantClient(path="data/qdrant_db")

# Normal


In [30]:
client.create_collection(
    collection_name="semantic_search",
    vectors_config=models.VectorParams(
        size=n_features, distance=models.Distance.COSINE
    ),
)

True

In [31]:
client.upsert(
    collection_name="semantic_search",
    points=data_to_insert,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [32]:
results = client.search(collection_name="semantic_search", query_vector=data[0])
for result in results:
    print(result)
truth_ids = [result.id for result in results]
truth_ids

id=0 version=0 score=0.9999999999710895 payload={'city': 'New York'} vector=None shard_key=None
id=79146 version=0 score=0.10068547854552665 payload={'city': 'Berlin'} vector=None shard_key=None
id=40476 version=0 score=0.09679997068001278 payload={'city': 'Berlin'} vector=None shard_key=None
id=79716 version=0 score=0.08865147490014064 payload={'city': 'New York'} vector=None shard_key=None
id=62986 version=0 score=0.08834369480462538 payload={'city': 'New York'} vector=None shard_key=None
id=92670 version=0 score=0.08665311739380321 payload={'city': 'New York'} vector=None shard_key=None
id=89988 version=0 score=0.08664402958882962 payload={'city': 'New York'} vector=None shard_key=None
id=8494 version=0 score=0.08639523257062481 payload={'city': 'New York'} vector=None shard_key=None
id=10977 version=0 score=0.08561782676244317 payload={'city': 'New York'} vector=None shard_key=None
id=21352 version=0 score=0.08395282853665643 payload={'city': 'Berlin'} vector=None shard_key=None


[0, 79146, 40476, 79716, 62986, 92670, 89988, 8494, 10977, 21352]

# Product Quantization


In [33]:
client.create_collection(
    collection_name="product_semantic_search",
    vectors_config=models.VectorParams(
        size=n_features, distance=models.Distance.COSINE
    ),
    quantization_config=models.ScalarQuantization(
        scalar=models.ScalarQuantizationConfig(
            type=models.ScalarType.INT8,
            quantile=0.99,  # 1% of extreme values will be excluded from quantization
            always_ram=True,
        ),
    ),
)

True

In [34]:
client.upsert(
    collection_name="product_semantic_search",
    points=data_to_insert,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
results = client.search(collection_name="product_semantic_search", query_vector=data[0])
predicted_ids = [result.id for result in results]
print(predicted_ids)
calculate_recall(truth_ids, predicted_ids)

[0, 79146, 40476, 79716, 62986, 92670, 89988, 8494, 10977, 21352]


1.0

# Binary Quantization


In [36]:
client.create_collection(
    collection_name="binary_semantic_search",
    vectors_config=models.VectorParams(
        size=n_features, distance=models.Distance.COSINE
    ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(
            always_ram=True,
        )
    ),
)

True

In [37]:
client.upsert(
    collection_name="binary_semantic_search",
    points=data_to_insert,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [38]:
results = client.search(collection_name="binary_semantic_search", query_vector=data[0])
predicted_ids = [result.id for result in results]
print(predicted_ids)
calculate_recall(truth_ids, predicted_ids)

[0, 79146, 40476, 79716, 62986, 92670, 89988, 8494, 10977, 21352]


1.0

# Time Metrics


In [39]:
table_times = {
    "semantic_search": [],
    "product_semantic_search": [],
    "binary_semantic_search": [],
}
n_reps = 15

for _ in range(n_reps):
    for table in table_times:
        query_vector = np.random.randn(1, n_features)[0]
        st_time = time.time()
        client.search(
            collection_name=table,
            query_vector=query_vector,
            search_params=models.SearchParams(
                quantization=models.QuantizationSearchParams(ignore=False)
            ),
        )
        end_time = time.time()
        table_times[table].append(end_time - st_time)

In [40]:
for table, times in table_times.items():
    print(table)
    print(f"Median: {np.median(times)}\n")

semantic_search
Median: 0.7421467304229736

product_semantic_search
Median: 0.7466821670532227

binary_semantic_search
Median: 0.7160227298736572

